# Chapter 5 - Parsing
파싱-훈련 데이터 분석하기

## 1 Parsing
파싱은 개별 구성요소에 대한 품사범주를 결정하고, 문법 규칙에 따르는지 여부를 분석

### 01 Tree bank corpus
트리뱅크 구성

In [6]:
import nltk
import nltk.corpus
print(str(nltk.corpus.treebank).replace('\\\\','/'))
print(nltk.corpus.treebank.fileids()[::50])

<BracketParseCorpusReader in '/home/markbaum/nltk_data/corpora/treebank/combined'>
['wsj_0001.mrg', 'wsj_0051.mrg', 'wsj_0101.mrg', 'wsj_0151.mrg']


In [3]:
from nltk.corpus import treebank
print(treebank.words('wsj_0007.mrg'))
print(treebank.tagged_words('wsj_0007.mrg'))

['McDermott', 'International', 'Inc.', 'said', '0', ...]
[('McDermott', 'NNP'), ('International', 'NNP'), ...]


In [7]:
# Treebank corpus 중 Penn Treebak corpus에 접속하는 코드내용 살펴보기
import nltk
from nltk.corpus import treebank
print(treebank.parsed_sents('wsj_0007.mrg')[2])

(S
  (NP-SBJ
    (NP (NNP Bailey) (NNP Controls))
    (, ,)
    (VP
      (VBN based)
      (NP (-NONE- *))
      (PP-LOC-CLR
        (IN in)
        (NP (NP (NNP Wickliffe)) (, ,) (NP (NNP Ohio)))))
    (, ,))
  (VP
    (VBZ makes)
    (NP
      (JJ computerized)
      (JJ industrial)
      (NNS controls)
      (NNS systems)))
  (. .))


In [10]:
%matplotlib inline
import nltk
from nltk.corpus import treebank_chunk
print(treebank_chunk.chunked_sents()[1])
treebank_chunk.chunked_sents()[1].draw()  # 외부 화면으로 출력

/home/markbaum/Python/python36/lib/python3.6/site-packages/nltk/tokenize/regexp.py:123: FutureWarning: split() requires a non-empty pattern match.
  return [tok for tok in self._regexp.split(text) if tok]


(S
  (NP Mr./NNP Vinken/NNP)
  is/VBZ
  (NP chairman/NN)
  of/IN
  (NP Elsevier/NNP N.V./NNP)
  ,/,
  (NP the/DT Dutch/NNP publishing/VBG group/NN)
  ./.)
